In [1]:
from utils import DATA
import config
import pdb

xdf_path = '/home/owais/EEGAnotator/Data/Patients/F10/PictureNaming/xdf/sample.xdf'
edf_path = '/home/owais/EEGAnotator/Data/Patients/F10/PictureNaming/eeg/sample.edf'



data = DATA(filepath_xdf=xdf_path, filepath_edf=edf_path)
data.print_info()

*********************************************************************************
***************************Loading .edf file***************************
Extracting EDF parameters from /home/owais/EEGAnotator/Data/Patients/F09/JIMÉNEZÁLVAREZ_605b2177-7b18-4d5b-a6df-01fdc6572770.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
*******************************Completed*******************************
Checking for Interruptions
No interruptions detected.
*********************************************************************************
***************************Loading .xdf file***************************
*******************************Completed*******************************
***************************EEG File Info***************************
Filepath: /home/owais/EEGAnotator/Data/Patients/F09/JIMÉNEZÁLVAREZ_605b2177-7b18-4d5b-a6df-01fdc6572770.edf
Start Time: 2022-04-26 17:58:00+00:00, End Time:2022-04-26 21:57:57.500000+00:00
Number of Channels: 14